In [1]:
import torch
import numpy as np
from tqdm import tqdm
from analytical_expressions import local_energy
from torch.autograd.functional import jacobian
from torch.func import jacrev
import matplotlib.pyplot as plt
from torch.func import vmap

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def psi(X):
    x = X[:3]
    y = X[3:6]
    alpha_1, alpha_2, alpha_3, alpha_4 = X[6:]
    r1 = torch.norm(x)
    r2 = torch.norm(y)
    r12 = torch.norm(x - y)

    term1 = torch.exp(-2 * (r1 + r2))
    term2 = 1 + 0.5 * r12 * torch.exp(-alpha_1 * r12)
    term3 = 1 + alpha_2 * (r1 + r2) * r12 + alpha_3 * (r1 - r2)**2 - alpha_4 * r12

    return term1 * term2 * term3

In [ ]:
L = 1
r1 = torch.rand(3, requires_grad=False) * 2 * L - L
r2 = torch.rand(3, requires_grad=False) * 2 * L - L #random number from -L to L
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True) # 1.013
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)
E = 0
E2 = 0
Eln_average = 0
ln_average = 0
rejection_ratio = 0
step = 0
max_steps = 500
N = 10000
dlap_dalpha = 0
inputs_arr = []
n_walkers = 5

for walkers in range(n_walkers):
    inputs = []
    for i in tqdm(range(N)):

        chose = np.random.rand()
        step = step + 1
        if chose < 0.5:
            r1_trial = r1 + 0.5 * (torch.rand(3) * 2 * L-L)
            r2_trial = r2
        else:
            r2_trial = r2 + 0.5 * (torch.rand(3) * 2 * L-L)
            r1_trial = r1


        X = torch.tensor(
            [*r1, *r2, alpha_1, alpha_2, alpha_3, alpha_4])
        X_trial = torch.tensor(
            [*r1_trial, *r2_trial, alpha_1, alpha_2, alpha_3, alpha_4])

        psi_val = psi(X)
        psi_trial_val = psi(X_trial)


        if psi_trial_val >= psi_val:
            r1 = r1_trial
            r2 = r2_trial
            
        else:
            dummy = np.random.rand()
            if dummy < psi_trial_val / psi_val:
                r1 = r1_trial
                r2 = r2_trial
            else:
                rejection_ratio += 1./N
                    
        if step > max_steps:
            
            X_final = torch.tensor(
            [*r1, *r2, alpha_1, alpha_2, alpha_3, alpha_4], requires_grad=True)

            #local_E = local_energy(X_final)
            #E += local_E / (N - max_steps)
            #dlap_dalpha += torch.autograd.grad(local_E, X_final, retain_graph=True)[0][6:] / (N - max_steps)
            inputs.append(X_final)

    inputs_arr.append(inputs)




In [4]:
psi_vec = vmap(psi)

In [21]:
def metropolis(N: int, n_runs: int, alphas: torch.tensor):  
    """
    Vectorized metropolis loop
    Over N steps, for n_runs. 
    Alphas passes in must be of same dim as n_runs
    """  
    assert alphas.shape[0] == n_runs        
    L = 1
    r1 = (torch.rand(n_runs, 3, requires_grad=True) * 2 * L - L)
    r2 = (torch.rand(n_runs, 3, requires_grad=True) * 2 * L - L)
    max_steps = 1000
    sampled_Xs = []
    rejection_ratio = 0

    for i in tqdm(range(N)):
        chose = torch.rand(n_runs).reshape(n_runs, 1)
        dummy = torch.rand(n_runs)

        perturbed_r1 = r1 + 0.5 * (torch.rand(n_runs, 3) * 2 * L - L)
        perturbed_r2 = r2 + 0.5 * (torch.rand(n_runs, 3) * 2 * L - L)

        r1_trial = torch.where(chose < 0.5, perturbed_r1, r1)
        r2_trial = torch.where(chose >= 0.5, perturbed_r2, r2)
        psi_val = psi_vec(torch.cat((r1, r2, alphas), axis=1))
        psi_trial_val = psi_vec(torch.cat((r1_trial, r2_trial, alphas), axis=1))      
        psi_ratio = psi_trial_val / psi_val

        density_comp = psi_trial_val >= psi_val
        dummy_comp = dummy < psi_ratio

        condition = density_comp + dummy_comp

        rejection_ratio += torch.where(condition, 1./N, 0.0)

        condition = condition.reshape(condition.shape[0], 1)

        # Careful with overwriting
        r1 = torch.where(condition, r1_trial, r1)
        r2 = torch.where(condition, r2_trial, r2)
                
        if i > max_steps:
            sampled_Xs.append(torch.cat((r1, r2, alphas), axis=1))

    return torch.stack(sampled_Xs)

In [22]:
local_e_vec = vmap(local_energy)
local_e_vec_vec = vmap(local_e_vec)

def get_local_energies(X):
    reshaped_X = X.reshape(
        X.shape[1], X.shape[0], X.shape[2])
    return local_e_vec_vec(reshaped_X)

def get_mean_energies(E):
    return torch.mean(E, dim=1)

In [23]:
def dE_dalpha(input):
    return jacrev(local_energy)(input)

dE_dalpha_vec = vmap(dE_dalpha)
dE_dalpha_vec_vec = vmap(dE_dalpha_vec)

def get_dE_dX(X):
    reshaped_X = X.reshape(
        X.shape[1], X.shape[0], X.shape[2])
    return dE_dalpha_vec_vec(X)

In [24]:
alpha_1 = torch.tensor(1.013, dtype=torch.float64, requires_grad=True) # 1.013
alpha_2 = torch.tensor(0.2119, dtype=torch.float64, requires_grad=True)
alpha_3 = torch.tensor(0.1406, dtype=torch.float64, requires_grad=True)
alpha_4 = torch.tensor(0.003, dtype=torch.float64, requires_grad=True)

In [30]:
alphas = torch.tensor([alpha_1, alpha_2, alpha_3, alpha_4]).unsqueeze(0).repeat(50, 1)

In [ ]:
sampled_Xs = metropolis(50000, 50, alphas=alphas)
# Metropolis is faster on cpu since we do a lot of operations back and forth I suppose? - When the overhead of transferring data to and from the GPU is greater than the benefits of parallel processing. There will be a point at which it'll flip but not currently

100%|██████████| 50000/50000 [01:20<00:00, 624.23it/s]


In [36]:
device = torch.device("cuda")
cpu = torch.device("cpu")

In [34]:
E = get_local_energies(sampled_Xs.to(device))

In [37]:
mean_E = get_mean_energies(E.to(cpu))

In [43]:
torch.mean(mean_E[:50])

tensor(-2.9382, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [44]:
sampled_Xs.shape

torch.Size([48999, 50, 10])

In [ ]:
for i in range(len(sampled_Xs))

In [45]:
gradients = get_dE_dX(sampled_Xs.to(device))

OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB. GPU 

In [194]:
reshaped_Xs = sampled_Xs.reshape(sampled_Xs.shape[1], sampled_Xs.shape[0], sampled_Xs.shape[2])

Make the rest of the code work with the super-vectorized approach

In [31]:
def dE_dalpha(input):
    return jacrev(local_energy)(input)

In [32]:
from torch.func import vmap

local_e_vec = vmap(local_energy)
dE_dalpha_vec = vmap(dE_dalpha)

energies = [local_e_vec(torch.stack(inputs_arr[i])) for i in range(len(inputs_arr))]

In [178]:
local_e_vec_vec = vmap(local_e_vec)

In [197]:
E_vec = local_e_vec_vec(reshaped_Xs)

In [202]:
torch.mean(torch.mean(E_vec, dim=0))

tensor(-2.6851, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [33]:
sum(energies[0])/(len(energies[0]))

tensor(-2.8161, dtype=torch.float64, grad_fn=<DivBackward0>)

In [9]:
sum([sum(energies[i])/len(energies[i]) for i in range(len(energies))])

tensor(-3.0107, dtype=torch.float64, grad_fn=<AddBackward0>)

Energy value should be −2.901188

The actual value is −2.9037243770

In [51]:
t = dE_dalpha_vec(torch.stack(inputs_arr[0]))

## Gradient values

In [34]:
def dE_dalpha(input):
    return jacrev(local_energy)(input)

t = dE_dalpha_vec(torch.stack(inputs_arr[0]))

In [72]:
dE_dalpha_mean = torch.mean(t, axis=0)

In [36]:
psi_vmap = vmap(psi)

In [38]:
psi_values = psi_vmap(torch.stack(inputs_arr[0]))

In [61]:
mean_energy = sum(energies[0])/(len(energies[0]))

In [63]:
El_Etheta = energies[0] - mean_energy

In [65]:
mean_psi = torch.mean(psi_values)

In [73]:
dE_dalpha_mean.shape

torch.Size([10])

In [86]:
t[0].shape

torch.Size([10])

In [89]:
psi_values.shape

torch.Size([9500])

In [92]:
psi_dalph = torch.stack([psi_values[i] * t[i] for i in range(len(t))])

In [97]:
psi_dalph.shape

torch.Size([9500, 10])

In [98]:
dE_dalpha_mean.shape

torch.Size([10])

In [95]:
mean_psi

tensor(0.0207, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [77]:
t.shape

torch.Size([9500, 10])

In [111]:
a = psi_values.unsqueeze(1).repeat(1, 10) * t

In [110]:
b = (mean_psi * dE_dalpha_mean).unsqueeze(0).repeat(9500, 1)

In [119]:
c = (energies[0] - mean_energy).unsqueeze(1).repeat(1, 10)

In [115]:
mean_energy.shape

torch.Size([])

In [120]:
gradients = (a - b) * (c)

In [124]:
torch.mean(gradients, axis=0)

tensor([ 0.0652,  0.4838,  0.4329,  0.0146,  0.1017,  0.0883,  0.0167, -0.0222,
        -0.0307,  0.0271], dtype=torch.float64, grad_fn=<MeanBackward1>)

In [52]:
energies[0][1]

tensor(-1.6312, dtype=torch.float64, grad_fn=<SelectBackward0>)

In [53]:
E_fixed = [energies[0][i] / psi_values[i] for i in range(len(inputs_arr[0]))]

In [55]:
torch.mean(torch.stack(E_fixed))

tensor(-2546.7772, dtype=torch.float64, grad_fn=<MeanBackward0>)